Code is used and adapted with permission mainly from 
- [Deploying a Simple Machine Learning Model into a WebApp using TensorFlow.js](https://towardsdatascience.com/deploying-a-simple-machine-learning-model-into-a-webapp-using-tensorflow-js-3609c297fb04) by Carlos Aguayo

Code is also used and adapted from
- [Handwritten Digit Recognition using Convolutional Neural Networks in Python with Keras](https://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-neural-networks-python-keras/) by Jason Brownlee

Data augmentation code is based on
- [Build a handwritten digit classifier app with TensorFlow Lite, Step 7: Improve model accuracy with data augmentation](https://colab.research.google.com/github/tensorflow/examples/blob/master/lite/codelabs/digit_classifier/ml/step7_improve_accuracy.ipynb#scrollTo=mxPxpHKHMAkl) by The Tensorflow Authors

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import pandas as pd
import random

print('tensorflow version', tf.__version__)

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Add a color dimension to the images in "train" and "validate" dataset to
# leverage Keras's data augmentation utilities later.
train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from keras.utils import np_utils

# Flatten 28*28 images to a 784 vector for each image
num_pixels = train_images.shape[1] * train_images.shape[2]
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1)).astype('float32')
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1)).astype('float32')

# One-hot encode outputs
train_labels = np_utils.to_categorical(train_labels)
test_labels = np_utils.to_categorical(test_labels)
num_classes = test_labels.shape[1]

# Define data augmentation
datagen = keras.preprocessing.image.ImageDataGenerator(
  rotation_range=30,
  width_shift_range=0.25,
  height_shift_range=0.25,
  shear_range=0.25,
  zoom_range=0.2)

datagen.fit(train_images)

# Define model
model = Sequential()

model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))
# The original code by Carlos Aguayo used 'softmax' in the output layer
# Here 'sigmoid' has been used instead for simplicity

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 200
epochs = 30

# Generate augmented data from MNIST dataset
train_generator = datagen.flow(train_images, train_labels, batch_size=batch_size)
test_generator = datagen.flow(test_images, test_labels, batch_size=batch_size)

# Fit model to augmented data
model.fit(train_generator, validation_data=test_generator, 
                    steps_per_epoch=len(train_images)/batch_size, epochs=epochs, verbose=2)

scores = model.evaluate(test_images, test_labels, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

In [ ]:
# Save whole model for download
model.save("model.h5")

In [ ]:
!pip install tensorflowjs

In [ ]:
!tensorflowjs_converter --input_format keras '/content/model.h5' '/content/model'